# Exact multiple sequence alignment with bounding.

![Description](Subject.png)

In [3]:
import numpy as np
from Bio.PDB import *
from Bio.Seq import Seq
import shutil
import warnings
import requests
import os
import pandas as pd
from Bio.PDB.PDBExceptions import PDBConstructionWarning
from Bio.PDB.DSSP import DSSP
from Bio import SeqIO
warnings.simplefilter('ignore', PDBConstructionWarning)

### Similarity Matrix

In [4]:
def load_blosum(filename):
    """
    Load a substitution matrix (e.g. BLOSUM) from a file.

    Returns:
        dict: {(aa1, aa2): score}
    """
    matrix = {}

    with open(filename) as f:
        lines = f.readlines()

    # Ignore comments
    lines = [line.strip() for line in lines if line.strip() and not line.startswith("#")]

    # First non-comment line = column headers
    headers = lines[0].split()

    # Remaining lines = rows
    for line in lines[1:]:
        parts = line.split()
        row_aa = parts[0]
        scores = parts[1:]

        for col_aa, score in zip(headers, scores):
            matrix[(row_aa, col_aa)] = int(score)

    return matrix

blosum = load_blosum("BLOSUM62.txt")

### Gap Parameters

In [5]:
gap = -4
gap_open = -11
gap_ext = -1

### Needleman-Wunsh algorithm for any MSA of K input sequences

Similarity column vs column

In [6]:
def score_column_vs_column(colA, colB, blosum, gap):
    total = 0
    count = 0

    for a in colA:
        for b in colB:
            # cas gap-gap
            if a == '-' and b == '-':
                continue

            # cas gap-aa
            if a == '-' or b == '-':
                total += gap
                count += 1
                continue

            # cas aa-aa
            if (a, b) in blosum:
                total += blosum[(a, b)]
            else:
                total += blosum[(b, a)]

            count += 1

    return total / count if count > 0 else 0.0


In [7]:
# Needleman-wunsh bloc-bloc affine

def needleman_wunsch_align_vs_align_affine(align1, align2, blosum, gap_open, gap_ext, gap_char="-"):
    n1, l1 = align1.shape
    n2, l2 = align2.shape
    NEG = -1e18

    # M: diag ; X: up (gap in align2) ; Y: left (gap in align1)
    matrix = np.full((l1+1, l2+1), NEG, dtype=float)
    X = np.full((l1+1, l2+1), NEG, dtype=float)
    Y = np.full((l1+1, l2+1), NEG, dtype=float)

    backtracking_matrix = np.full((l1+1, l2+1), -1, dtype=np.int8)  # prev state 0/1/2
    btX = np.full((l1+1, l2+1), -1, dtype=np.int8)
    btY = np.full((l1+1, l2+1), -1, dtype=np.int8)

    matrix[0,0] = 0.0
    backtracking_matrix[0,0] = 0
    # 1 = horiz ; 2 = vert ; 3 = diag

    # ---- CONSTRUCTION MATRICES ----
    # init first column (only X)
    for i in range(1, l1+1):
        if i == 1:
            X[i,0] = matrix[i-1,0] + gap_open + gap_ext
            btX[i,0] = 0
        else:
            X[i,0] = X[i-1,0] + gap_ext
            btX[i,0] = 1
            matrix[i,0] = NEG

    # init first row (only Y)
    for j in range(1, l2+1):
        if j == 1:
            Y[0,j] = matrix[0,j-1] + gap_open + gap_ext
            btY[0,j] = 0
        else:
            Y[0,j] = Y[0,j-1] + gap_ext
            btY[0,j] = 2
            matrix[0,j] = NEG


    for i in range(1,l1+1):
        for j in range(1,l2+1):                
            # X (up): either open from M or extend X
            open_x = matrix[i-1,j] + gap_open + gap_ext
            ext_x  = X[i-1,j] + gap_ext
            if open_x >= ext_x:
                X[i,j] = open_x
                btX[i,j] = 0
            else:
                X[i,j] = ext_x
                btX[i,j] = 1

            # Y (left): open from M or extend Y
            open_y = matrix[i,j-1] + gap_open + gap_ext
            ext_y  = Y[i,j-1] + gap_ext
            if open_y >= ext_y:
                Y[i,j] = open_y
                btY[i,j] = 0
            else:
                Y[i,j] = ext_y
                btY[i,j] = 2

            # M (diag): come from best previous state + column score
            s = score_column_vs_column(align1[:, i-1], align2[:, j-1], blosum, 0) #modifier score pour pas compter les gap 2 fois
            prev = [matrix[i-1,j-1], X[i-1,j-1], Y[i-1,j-1]]
            p = int(np.argmax(prev))
            matrix[i,j] = prev[p] + s
            backtracking_matrix[i,j] = p


    # choose best ending state
    i, j = l1, l2
    end = [matrix[i,j], X[i,j], Y[i,j]]
    state = int(np.argmax(end))
    score = end[state]

    # ---- BACKTRACKING ----
    cols1 = []
    cols2 = []

    i, j = l1, l2
    while i > 0 or j > 0:
        if state == 0:  # M: diag
            prev = backtracking_matrix[i,j]
            cols1.append(align1[:, i-1])
            cols2.append(align2[:, j-1])
            i -= 1; j -= 1
            state = int(prev)
        elif state == 1:  # X: up (gap in align2)
            prev = btX[i,j]
            cols1.append(align1[:, i-1])
            cols2.append(np.array([gap_char]*n2))
            i -= 1
            state = int(prev)
        else:  # Y: left (gap in align1)
            prev = btY[i,j]
            cols1.append(np.array([gap_char]*n1))
            cols2.append(align2[:, j-1])
            j -= 1
            state = int(prev)

    cols1.reverse()
    cols2.reverse()

    new1 = np.stack(cols1, axis=1)  # (n1, L)
    new2 = np.stack(cols2, axis=1)  # (n2, L)
    final_align = np.vstack([new1, new2])  # (n1+n2, L)     

    return new1, new2, matrix, score, final_align

In [8]:
#TEST ALIGN VS ALIGN AFFINE
print("========== TEST ALIGN VS ALIGN AFFINE ======")
align1 = np.array([
    list("CHAT"),
    list("C-AT")])

align2 = np.array([
    list("BAT"),
    list("CAT")])

new_align1, new_align2 , M, score, final_align = needleman_wunsch_align_vs_align_affine(align1, align2, blosum, gap_open, gap_ext, gap_char='-')

print("score:", score)
print(M)
print("new align:")
for row in final_align:
    print("".join(row))

========== TEST ALIGN VS ALIGN AFFINE ======
score: 0.0
[[ 0.000e+00 -1.000e+18 -1.000e+18 -1.000e+18]
 [-1.000e+18  3.000e+00 -1.200e+01 -1.400e+01]
 [-1.000e+18 -1.275e+01  2.000e+00 -1.000e+01]
 [-1.000e+18 -1.400e+01 -5.000e+00  2.000e+00]
 [-1.000e+18 -1.500e+01 -1.000e+01  0.000e+00]]
new align:
CHAT
C-AT
B-AT
C-AT
